In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/justice-ai-case-dataset/justice_ai_dataset.csv


# 🎯 Goal:
We implemented a custom priority scoring logic for Justice-AI to identify which cases should be addressed first.

# ⚖️Highlights:
Rape, Murder, and Kidnapping get the highest base scores.

Media Attention, Pending Status, and Strong Evidence boost priority.

Older cases gain more weight over time.

# 📊 Output:
Top 10 highest-priority cases are shown based on calculated scores.

In [6]:
import pandas as pd
df = pd.read_csv("/kaggle/input/justice-ai-case-dataset/justice_ai_dataset.csv")

In [7]:
import pandas as pd
from datetime import datetime

def calculate_priority_score(row):
    score = 0

    # High priority case types first
    if row['case_type'] == 'Rape':
        score += 1000
        if row['status'] == 'Pending':
            score += 80
        elif row['status'] == 'Under Trial':
            score += 60
        if row['media_attention']:
            score += 70
        if row['evidence_available']:
            score += 50

    elif row['case_type'] == 'Kidnapping':
        score += 900
        if row['status'] == 'Pending':
            score += 80
        elif row['status'] == 'Under Trial':
            score += 60
        urgency_score = {'High': 60, 'Medium': 40, 'Low': 20}
        score += urgency_score.get(row['urgency_level'], 0)
        if row['media_attention']:
            score += 50
        if row['evidence_available']:
            score += 40

    elif row['case_type'] == 'Murder':
        score += 850
        if row['status'] == 'Pending':
            score += 70
        elif row['status'] == 'Under Trial':
            score += 50
        if row['media_attention']:
            score += 50
        if row['evidence_available']:
            score += 40

    elif row['case_type'] == 'Accidents':
        score += 800
        score += row['victims_count'] * 10
        severity_score = {'High': 40, 'Medium': 20, 'Low': 10}
        score += severity_score.get(row['severity_level'], 0)
        if row['status'] == 'Pending':
            score += 50
        elif row['status'] == 'Under Trial':
            score += 30

    elif row['case_type'] == 'Theft':
        score += 700
        urgency_score = {'High': 60, 'Medium': 40, 'Low': 20}
        score += urgency_score.get(row['urgency_level'], 0)
        if row['status'] == 'Pending':
            score += 50
        elif row['status'] == 'Under Trial':
            score += 30

    elif row['case_type'] == 'Domestic Violence':
        score += 650
        score += row['victims_count'] * 10
        if row['status'] == 'Pending':
            score += 50
        elif row['status'] == 'Under Trial':
            score += 30

    elif row['case_type'] == 'Corruption':
        score += 600
        if row['media_attention']:
            score += 50
        score += row['victims_count'] * 5
        if row['status'] == 'Pending':
            score += 50
        elif row['status'] == 'Under Trial':
            score += 30

    elif row['case_type'] == 'Cyber Crime':
        score += 500
        score += row['victims_count'] * 10
        if row['status'] == 'Pending':
            score += 50
        elif row['status'] == 'Under Trial':
            score += 30

    elif row['case_type'] == 'Defamation':
        score += 300
        if row['status'] == 'Pending':
            score += 30
        elif row['status'] == 'Under Trial':
            score += 20

    elif row['case_type'] == 'Land Disputes':
        score += 100
        if row['status'] == 'Pending':
            score += 30
        elif row['status'] == 'Under Trial':
            score += 20

    # bonus score for older cases (older date = higher score)
    try:
        filed_date = pd.to_datetime(row['date_filed'], errors='coerce')
        if pd.notnull(filed_date):
            days_pending = (datetime.now() - filed_date).days
            score += int(days_pending / 30)  # More months = more priority
    except:
        pass

    return score


In [11]:
df['priority_score'] = df.apply(calculate_priority_score, axis=1)
df_sorted = df.sort_values(by='priority_score', ascending=False)


In [15]:
df_sorted[['case_id', 'case_type', 'priority_score']].head(10)


,case_id,case_type,priority_score
3,C004,Rape,1225
41,C042,Kidnapping,1131
20,C021,Kidnapping,1130
27,C028,Kidnapping,1125
29,C030,Rape,1124
12,C013,Kidnapping,1113
1,C002,Rape,1098
36,C037,Rape,1084
25,C026,Murder,1077
21,C022,Kidnapping,1067
